# Working with arrays of strings in Blosc2

Blosc2 provides support for arrays in which the elements are strings, either of bytes (``np.bytes_`` equivalent to ``np.dtype('S0')``) or of unicode characters (``np.str_``, equivalent to ``np.dtype('U0')``), with the typesize determined by the longest element in the array. That is

In [1]:
import numpy as np

arr = np.array([b"a23", b"89u"])
print(f"Bytes array - dtype: {arr.dtype}, typesize: {arr.dtype.itemsize}")
arr = np.array(["a23", "89u"])
print(f"Unicode array - dtype: {arr.dtype}, typesize: {arr.dtype.itemsize}")

Bytes array - dtype: |S3, typesize: 3
Unicode array - dtype: <U3, typesize: 12



since each unicode character encodes 4 bytes. This carries over to the ``blosc2.NDArray`` object. Indeed, such arrays, particularly those of Unicode type, are highly compressible, since almost all of the bits encoding each item will be 0 (i.e. ``\x00``), as can be seen by viewing the second array above as an array of bytestrings

In [2]:
arr.view("S12")

array([b'a\x00\x00\x002\x00\x00\x003', b'8\x00\x00\x009\x00\x00\x00u'],
      dtype='|S12')

(The trailing ``\x00`` bytes are suppressed for the last character). Consequently, using Blosc2 can save you a lot of space (in memory or disk) when working with arrays of strings, if one exploits the structure of the (unicode) strings correctly. Specifically, the fundamental building block of the array should be the byte, and not the element - in this way, using the shuffle filter groups the $N$ elements having $m$ characters of bytesize 4 into $4$ streams of $Nm$ bytes, so that the corresponding bytes for all characters are grouped together. For the array above, one transforms the array from (2 elements of $3 \times 4 = 12$ bytes)
```
|a\x00\x00\x002\x00\x00\x003\x00\x00\x00|8\x00\x00\x009\x00\x00\x00u\x00\x00\x00|
```
to (4 streams of $2 \times 3 = 6$ bytes)
```
|a2389u|\x00\x00\x00\x00\x00\x00|\x00\x00\x00\x00\x00\x00|\x00\x00\x00\x00\x00\x00|
```
For the example above, 3 of the bytes for each character are 0, and so by grouping these zeros together, it is more likely to have chunks composed entirely or almost entirely of zeros, which may then be readily compressed.
If one were to break up the array by elements, one would end up with $4m$ streams of $N$ bytes, i.e. ($4 \times 3 = 12$ streams of $2$ bytes)
```
|a8|\x00\x00|\x00\x00|\x00\x00|29|\x00\x00|\x00\x00|\x00\x00|3u|\x00\x00|\x00\x00|\x00\x00|
```
which is not very compressible, since the informative bytes are fragmented by small groups of 0 bytes. This heuristic has been implemented as a default for Blosc2 string compression, so you can just compress your arrays of strings and reap the benefits without having to worry about such intricacies though! Check it out below:

In [3]:
import blosc2

N = int(1e5)
nparr = np.repeat(np.array(["josé", "pepe", "francisco"]), N)
cparams = blosc2.cparams_dflts
arr1 = blosc2.asarray(nparr, cparams=cparams)
print(f"cratio forcing non-string defaults: {round(arr1.cratio)}x")
arr1 = blosc2.asarray(nparr)
print(f"cratio allowing blosc2 to optimise: {round(arr1.cratio)}x")

cratio forcing non-string defaults: 2417x
cratio allowing blosc2 to optimise: 3902x


## Operating on arrays of strings

Blosc2 has two sides, compression and computation, which are tightly enmeshed, and for arrays of strings the same applies. We have implemented a subset of useful functions for strings:
- comparison operations ``<, <=, ==, !=, >=, >``
- 2-argument functions ``contains, startswith, endswith``
- 1-argument functions ``lower, upper``

Where possible these will be computed by the ``miniexpr`` backend, which is a highly optimised, fully compiled, multithreaded library which is the most complete expression of Blosc2's goal: fully vertically integrated decompression/computation/recompression with optimal cache hierarchy exploitation and super-fast compiled-C code for as much of the pipeline as possible. In cases where this is not possible, a more robust path which still avoids memory overload for large arrays is used.

The arguments may be scalars or arrays (``blosc2.NDArray`` or other types) of strings or bytes. 

In [4]:
for t in ("bytes", "string"):
    if t == "bytes":
        a1 = np.array([b"abc", b"def", b"aterr", b"oot", b"zu", b"ab c"])
        a2 = a2_blosc = b"a"
    else:
        a1 = np.array(["abc", "def", "aterr", "oot", "zu", "ab c"])
        a2 = a2_blosc = "a"
    a1_blosc = blosc2.asarray(a1)
    for func, npfunc in zip(
        (blosc2.startswith, blosc2.endswith, blosc2.contains),
        (np.char.startswith, np.char.endswith, lambda *args: np.char.find(*args) != -1),
        strict=True,
    ):
        expr_lazy = func(a1_blosc, a2_blosc)
        res_numexpr = npfunc(a1, a2)
        assert expr_lazy.shape == res_numexpr.shape
        assert expr_lazy.dtype == blosc2.bool_
        np.testing.assert_array_equal(expr_lazy[:], res_numexpr)

    np.testing.assert_array_equal((a1_blosc < a2_blosc)[:], a1 < a2)
    np.testing.assert_array_equal((a1_blosc <= a2_blosc)[:], a1 <= a2)
    np.testing.assert_array_equal((a1_blosc == a2_blosc)[:], a1 == a2)
    np.testing.assert_array_equal((a1_blosc != a2_blosc)[:], a1 != a2)
    np.testing.assert_array_equal((a1_blosc >= a2_blosc)[:], a1 >= a2)
    np.testing.assert_array_equal((a1_blosc > a2_blosc)[:], a1 > a2)

    for func, npfunc in zip((blosc2.lower, blosc2.upper), (np.char.lower, np.char.upper), strict=True):
        expr_lazy = func(a1_blosc)
        res_numexpr = npfunc(a1)
        assert expr_lazy.shape == res_numexpr.shape
        np.testing.assert_array_equal(expr_lazy[:], res_numexpr)